# Importing AIS and CERS data

This code assumes you have access to two sets of data, Automatic Identification System (AIS) and Consolidated European Reporting System (CERS) as described below.

In [1]:
# base libraries
import numpy as np
import pandas as pd
import datetime
import glob
import os
import json

In [2]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

ais_path = config['DEFAULT']['ais_path']
ais_filename = config['DEFAULT']['ais_filename']
cers_path = config['DEFAULT']['cers_path']
port_filename = config['DEFAULT']['port_filename']
vessel_filename = config['DEFAULT']['vessel_filename']
ship_class_path = config['DEFAULT']['ship_class_path']
ship_class_filename = config['DEFAULT']['ship_class_filename']
processing_path = config['DEFAULT']['processing_path']
shipping_filename = config['DEFAULT']['shipping_filename']
ais_op_filename1 = config['DEFAULT']['ais_op_filename1']
ais_op_filename2 = config['DEFAULT']['ais_op_filename2']
ais_op_filename3 = config['DEFAULT']['ais_op_filename3']
ais_op_filename4 = config['DEFAULT']['ais_op_filename4']

## AIS Data

The Automatic Identification System (AIS) transmits data including speed, position, rate of turn, bearing and so on. This data allows ships to view traffic in their local area (10-20 nautical miles) and to simultaneously be seen by that traffic.

The AIS system is used to support:
- collision avoidance, notably amongst vessels outside the range of shore-based systems
- fishing fleet monitoring and control
- vessel traffic services, used to augment existing systems such as local vessel traffic service (VTS)
- maritime security, indentify and monitor suspicious activity patterns
- aids to navigation, AIS may support or replace information generated by radar beacons currently used for electronic navigation aids
- search and rescue, coordinating on-scene resources of a marine search and rescue (SAR) operation
- accident investigation, AIS information is more accurate and comprehensive than transitional systems such as radar
- ocean current estimates
- fleet and cargo tracking

There are two main types of AIS message.  Class A messages are sent by large ships typically over 300 tonnes of those carrying passengers.  Class B messages are used by lighter commercial and leisure craft.  In both cases, an AIS transmitter will send the following information every two to 10 seconds when underway and every three minutes when at anchor:
- the vessel's Maritime Mobile Service Identity (MMSI), the unique ship identifier
- navigation status, at anchor, under way using engine(s), not under command and so on
- rate of turn, right or left, from 0 to 720 degrees per minute
- speed over ground, 0.1 knot resolution from 0 to 102 knots 
- longitude, accurate to 0.0001 minutes
- latitude, accurate to 0.0001 minutes
- course over ground, relative to true north to 0.1°
- true heading, 0 to 359 degrees 
- true bearing at own position, 0 to 359 degrees
- UTC Seconds, the seconds field of the UTC time when the data were generated

A recent development of the AIS system is the ability to make it viewable publically, on the internet, without the need for a dedicated receiver. This removes the range limitation of the marine based receivers and allows AIS data to be visualised over an unlimited area. With permission, data may also be downloaded to be used for additional offline processing.  

After processing of AIS messages as described here (), the AIS dataset contains one row per ship per AIS message, 
it gives both static (latitude, longitude) and dynamic (speed, rate of turn) information at every point
where the ships AIS transponder transmits a message to a receiving base station. The AIS dataset contains the following columns:

|Column|Format|Description|
|---|---|---|
|MMSI|Numeric|Unique identifier of a ship|
|dt|datetime|Timestamp of AIS message|
|lat|decimal|Latitude of ship at timestamp|
|long|decimal|Longitude of ship at timestamp|
|NavStat|decimal||
|SOG|decimal|Speed over ground of ship at timestamp|
|HDG|decimal|Heading of ship at timestamp|
|COG|decimal|Course over ground at timestamp|
|ROT|decimal|Rate of turn of ship at timestamp|

In [3]:
# import AIS data
ais = pd.read_csv(ais_path+ais_filename,header = 0,delimiter = ',')

In [4]:
# ais['dt'] = pd.to_datetime(ais["time"], unit='s')
ais['dt'] = pd.to_datetime(ais["dt"])

In [5]:
# duplicates can occur when more than one base station picks up a signal
ais.drop_duplicates(['MMSI','dt'], keep = 'first', inplace = True)

## CERS Data
The Consolidated European Reporting System (CERS) is used by masters, shipping agents and port authorities to provide mandatory reportable information when a vessel arrives at a port in the UK.  It captures ship arrival and departure notifications, dangerous or polluting goods notifications, and notifications of port waste infringements and bulk carrier infringements, for all the ports within UK waters.

A report must be made at least 24 hours in advance of arrival or departure for the following:
- all ships carrying passengers
- all ships of 300 gross tonnage and above, when bound for a UK port
- all recreational craft of 45 metres length and over, when bound for a UK port
- all ships regardless of size, when carrying dangerous or polluting goods, either departing from or bound to a UK port

The CERS dataset is downloaded as three separate files containing information on:

Ports - contains one row per port, giving their LOCODE (internally recognised code for a port), name and size of port

Vessels - contains one row per ship, giving their unique identifier (MMSI), name and gross tonnage

Voyages - contains one row per voyage, giving the associated ship (MMSI), expected time of arrival, port of call, previous port of call, next port of call and information on hazardous materials

Note only the relevant variables are decribed above.

### Import port data

Data from CERS gives details of ports such as unique identifier, name, port size, voyage count  

In [6]:
port = pd.read_csv(cers_path+port_filename,header = 0,delimiter = ',')

In [7]:
port.rename(columns = {'?Id':'id','Name':'name','Port Size':'port_size','Voyage Count':'voyage_count'},inplace = True)

### Import vessels data

Data from CERS gives details of the vessels such as vessel name and gross tonnage.

In [8]:
vessel = pd.read_csv(cers_path + vessel_filename,header = 0,delimiter = ',')

In [9]:
vessel.rename(columns = {'?Vessel Id':'vessel_id','Vessel Name':'vessel_name','Callsign':'callsign',
                         'Gross Tonnage':'gross_tonnage','Last Voyage Id':'last_voyage_id',
                         'Last Voyage Created':'last_voyage_created','Last Port of Call':'last_POC',
                         'Last Voyage ETA':'last_voyage_ETA','Last Voyage ETD':'last_voyage_ETD'},inplace = True)

In [10]:
vessel['vessel_name'].fillna('',inplace = True)
vessel['IMO'].fillna('-1',inplace = True)
vessel['MMSI'].fillna('-1',inplace = True)
vessel['callsign'].fillna('',inplace = True)
vessel['gross_tonnage'].fillna('0',inplace = True)
vessel['last_voyage_id'].fillna('-1',inplace = True)
vessel['last_voyage_created'].fillna(pd.Timestamp.min,inplace = True)
vessel['last_POC'].fillna('',inplace = True)

In [11]:
vessel['IMO'] = vessel.IMO.astype(int)
vessel['MMSI'] = vessel.MMSI.astype(int)
vessel['gross_tonnage'] = vessel.gross_tonnage.astype(int)
vessel['last_voyage_id'] = vessel.last_voyage_id.astype(int)
vessel['last_voyage_created'] = pd.to_datetime(vessel['last_voyage_created'])
vessel['last_voyage_ETA'] = pd.to_datetime(vessel['last_voyage_ETA'])
vessel['last_voyage_ETD'] = pd.to_datetime(vessel['last_voyage_ETD'])

### Import voyages data

Data from CERS gives information on each of the ships voyages where they involve a UK port. The data includes details of the journey such as port of call, expected time of arrival (ETA), actual time of arrival (ATA), previous port and whether the vessel is carrying hazadous materials. The ETA and ATA are vital information for calculating the delay of a ship arriving into port.

In [12]:
# concatenate all the individual voyage datasets that match the naming convention defined by fnpattern
fnpattern = '*_voyage_*.csv'
all_files = glob.glob(os.path.join(cers_path,fnpattern))
df_from_each_file = (pd.read_csv(f, header=0, delimiter=',') for f in all_files)
voyage = pd.concat(df_from_each_file, ignore_index=True)

In [13]:
voyage.rename(columns = {'?Voyage ID':'voyage_id','Voyage Status':'voyage_status','Port of Call LOCODE':'POC_LOCODE',
                         'Vessel Name':'vessel_name','Last Port LOCODE':'last_port_LOCODE','Next Port LOCODE':'next_port_LOCODE',
                         'In Msg Create Date':'in_msg_create_dt','Out Msg Create Date':'out_msg_create_dt',
                         'Incoming Hazmat On Board':'in_hazmat','Outgoing Hazmat On Board':'out_hazmat'}, inplace = True)
voyage = voyage[['voyage_id','MMSI','ETA','ETD','ATA','ATD','last_port_LOCODE','next_port_LOCODE','in_hazmat','out_hazmat','POC_LOCODE']]

In [14]:
voyage['last_port_LOCODE'].fillna('',inplace = True)
voyage['next_port_LOCODE'].fillna('',inplace = True)
voyage['MMSI'].fillna('-1',inplace = True)
voyage['in_hazmat'].fillna('',inplace = True)
voyage['out_hazmat'].fillna('',inplace = True)

In [15]:
voyage['voyage_id'] = voyage.voyage_id.astype(int)
voyage['ETA'] = pd.to_datetime(voyage['ETA'])
voyage['ETD'] = pd.to_datetime(voyage['ETD'])
voyage['ATA'] = pd.to_datetime(voyage['ATA'])
voyage['ATD'] = pd.to_datetime(voyage['ATD'])
voyage['MMSI'] = voyage.MMSI.astype(int)

In [16]:
# depending on how the exports of data from CERS were split (e.g. by date) it's possible for duplicates 
# to exist in the voyage data
voyage.drop_duplicates(['MMSI','ETA'], keep = 'first', inplace = True)

### Import ship classes data
Data containing the class of each ship (as defined by MMSI), this data was obtained by manually searching the information on the internet. 

In [17]:
ship_class = pd.read_csv(ship_class_path+ship_class_filename,header = 0,delimiter = ',')

## Merge AIS and CERS data

The AIS data gives a continuous set of GPS points for the time that the AIS equipment was operating. However these GPS points are not organised into separate voyages. By combining the two datasets we have one dataset with the information on voyages and arrival times, plus the information which can be used to derive features about the ships' behaviour from the GPS locations.

In [18]:
# join ship_class to AIS data by MMSI
ais = ais.merge(ship_class, how = 'left', on = 'MMSI')

In [19]:
# join vessels data to AIS data
ais = ais.merge(vessel[['MMSI','gross_tonnage','vessel_name']], how = 'left', on = 'MMSI')

The only common identifier between the AIS and CERS data is the MMSI. We use MMSI to join the two datasets together then restrict each datetime (dt) within the AIS to it's closest ETA or ETD.

In [20]:
# join voyage data to AIS data
# restrict AIS data to intersection with voyage data
ais = ais.merge(voyage,how = 'inner',on = 'MMSI', sort = True)
# retain records where the AIS point falls within 24 hours of the ETA or ETD window
mask = (ais['dt'] < ais['ETD'] + datetime.timedelta(seconds = 86400)) & (ais['dt'] > ais['ETA'] - datetime.timedelta(seconds = 86400))
ais = ais[mask] 

In [21]:
# keep only the closest ETA or ETD
ais['ETA_dist'] = abs(ais['ETA'] - ais['dt'])
ais['ETD_dist'] = abs(ais['ETD'] - ais['dt'])
ais['ET_min'] = ais[['ETA_dist','ETD_dist']].min(axis=1)
ais_grouped = ais.groupby(['MMSI','dt'], as_index = False)['ET_min'].min()
ais = ais.merge(ais_grouped, how = 'inner')
ais.drop(['ETA_dist','ETD_dist','ET_min'],axis=1, inplace=True)

In [22]:
# remove any remaining duplicates
ais.drop_duplicates(['MMSI','dt'], keep = 'first', inplace = True)

## Data from outside the port

Due to size, the data from outside the port is imported separately and appended to the in port data

In [23]:
ais1 = pd.read_csv(ais_path+ais_op_filename1,header = 0,delimiter = ',')
ais2 = pd.read_csv(ais_path+ais_op_filename2,header = 0,delimiter = ',')
ais3 = pd.read_csv(ais_path+ais_op_filename3,header = 0,delimiter = ',')
ais4 = pd.read_csv(ais_path+ais_op_filename4,header = 0,delimiter = ',')

ais1['dt'] = pd.to_datetime(ais1["time"], unit='s')
ais2['dt'] = pd.to_datetime(ais2["time"], unit='s')
ais3['dt'] = pd.to_datetime(ais3["time"], unit='s')
ais4['dt'] = pd.to_datetime(ais4["time"], unit='s')

ais_op = ais1.append(ais2)
ais_op = ais_op.append(ais3)
ais_op = ais_op.append(ais4)

ais_op = ais_op.dropna(subset=['MMSI'])

In [24]:
ais_op = ais_op.sort_values(by=['MMSI','dt'])

# duplicates can occur if the lats and longs of the four files overlap
ais_op.drop_duplicates(['MMSI','dt'], keep = 'first', inplace = True)
ais_op.drop(['time','TimeStamp','ship'],axis = 1,inplace=True)

In [25]:
# merge ship class and vessel data
ais_op = ais_op.merge(ship_class, how = 'left', on = 'MMSI')
ais_op = ais_op.merge(vessel[['MMSI','gross_tonnage','vessel_name']], how = 'left', on = 'MMSI')

In [34]:
# append the out of port data to the in port data
ais['status'] = 'In Port'
ais_op['status'] = 'Outside Port'
ais_all = pd.concat([ais,ais_op])

In [35]:
# drop duplicates between in port and out of port data
ais_all.drop_duplicates(['MMSI','dt'], keep = 'first', inplace = True)

### Tripid

Add a tripid to identify different voyages. Due to the fact operators choose to turn the AIS equipment on and off and different times and the condition of the data it's not possible to derive an exact tripid. Some operators leave the equipment on when docked in a port, some don't turn the equipment on when they first leave the port. Therefore some tripids will cover the whole voyage, inbound and outbound, some will only cover a leg of the voyages.

The tripid is derived by incrementing a value by 1 each time there is a gap of more than 14,440 seconds (4 hours).

In [36]:
# sort the data first to make sure it's ordered by unique ship (MMSI) and timepoint
ais_all = ais_all.sort_values(by=['MMSI','dt'])
# shift the datetime by one and calculate the difference between the datetime and the previous timepoint
ais_all['prev_dt'] = ais_all.groupby(['MMSI'])['dt'].shift(1)
ais_all['date_diff'] = ais_all['dt'] - ais_all['prev_dt']
ais_all['date_diff_secs'] = ais_all['date_diff'].dt.total_seconds()

# when the difference is greater than than 14400 seconds (4 hours) increment a counter by one
ais_all['tripid'] = (ais_all['date_diff_secs'] > 14400).groupby(ais_all['MMSI']).cumsum()
ais_all['tripid'] = ais_all['tripid'].astype(int)+1

In [37]:
# only interested in journeys with AIS points both outside and inside the port
ais_status = ais_all.groupby(['MMSI','tripid','status'],as_index = False)['dt'].first()
ais_status = ais_status.groupby(['MMSI','tripid'],as_index = False)['status'].count()
ais_both = ais_status[ais_status.status > 1]

ais_all = ais_all.merge(ais_status[['MMSI','tripid']],how = 'inner', on = ['MMSI','tripid'])

In [38]:
# The out of port data is not complete for ETA, POC_LOCODE, in_hazmat and out_hazmat. As these variables do not 
# change across the voyage they can be imputed from the in port data. These variables are filled forwards for 
# data after the ship leaves the port and back filled for data before the ship entered the port
ais_all['ETA'] = ais_all.groupby(['MMSI','tripid'], sort=False)['ETA'].apply(lambda x: x.ffill().bfill())
ais_all['POC_LOCODE'] = ais_all.groupby(['MMSI','tripid'], sort=False)['POC_LOCODE'].apply(lambda x: x.ffill().bfill())
ais_all['in_hazmat'] = ais_all.groupby(['MMSI','tripid'], sort=False)['in_hazmat'].apply(lambda x: x.ffill().bfill())
ais_all['out_hazmat'] = ais_all.groupby(['MMSI','tripid'], sort=False)['out_hazmat'].apply(lambda x: x.ffill().bfill())

In [39]:
ais_all.drop(['ATA','ATD','COG','ETD','HDG','NavStat','prev_dt','date_diff','date_diff_secs'], axis = 1, inplace=True)

## Exporting Data

In [40]:
ais_all.to_csv(processing_path + shipping_filename,header=True,index=False,sep=',')

In [41]:
ais_all.describe().astype(int)

,MMSI,ROT,SOG,gross_tonnage,lat,long,voyage_id,tripid
count,12287388,12287388,12287388,6608557,12287388,12287388,2168734,12287388
mean,278546083,129,14,36367,51,1,1086327,104
std,110241080,439,8,49744,0,0,36605,119
min,41491304,-713,0,0,51,1,1001116,1
25%,235078345,0,7,2998,51,1,1057340,6
50%,235099364,0,14,16342,51,1,1087894,52
75%,246171000,713,23,53208,51,1,1114557,183
max,970013941,719,102,210000,51,1,1164010,693
